In [1]:
import torch
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
else:
    print('Cuda not available')

Cuda not available


In [19]:
from FaultsClass import *
import glob
import fnmatch
import os
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

source_path = 'C:/Users/Student/Desktop/Konduri/s/'    # specifying the path for accessing the data
current_path = os.getcwd()   # specifying the current path for saving the data
output_path = '/MainDataset'   # generate folder to save output date in it

ext = 'jpg' # image extension, for different extension like png this should modified
faults = ['Blur', 'Rain', 'SpeckleNoise'] # list of fault will be generated, each in its folder
intensity = ['l', 'm', 'e']   # list of intensity will be generated, each for each fault folders
seperator = '_'  # used in saving image with name contains '_' as seperator for image name, fault type and intensiy information

fields = ['id', 'fault type', 'intensity', 'c_Path', 'f_Path'] # header (the first row) of the csv file

def countImagesInFolder(path, ext = ext):
    print('found ',len(fnmatch.filter(os.listdir(path), '*.'+ ext)), 'file(s) with the extenstion ".{}"'.format(ext),'\n')
    
def createDirectory(folder_name, path = os.getcwd()):
    folder_path = path +'/'+ folder_name
    # Check whether the specified path exists or not
    isExist = os.path.exists(folder_path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(folder_path)
        print("The new directory "+ folder_name +" is created!")
    else:
        print("The directory "+ folder_name +" is already EXIST!",'Data in folder may get replaced')        
    
def createFolders(faults_list, intensity_list, output_folder = True, no_fault = True):
    if output_folder:
        createDirectory(output_path, current_path) # create output folder inside curret directory
    if no_fault:
        createDirectory('NoFault', current_path+ '/' +output_path) # create no fault folder inside curret directory

    # create faults folders inside output folder
    for fault in faults_list:
        createDirectory(fault, current_path +'/' + output_path)
        # create intensity folders inside output folder
        for item in intensity_list:
            createDirectory(item, current_path +'/' + output_path + '/' + fault)

def specifyImageName(iteration, fault_type, fault_intensity = '', ext = '.jpg'):
    if fault_type != 'NoFault':
        return str(iteration) + seperator + fault_type + seperator + fault_intensity + ext
    else:
        return str(iteration) + seperator + fault_type + ext

def saveImage(path, image_name, image):
    im_rgb = cv2.cvtColor(asarray(image), cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(path, image_name), im_rgb)
    
def saveAllFaults(source_path, new_size = (224, 224, 3)):
    i = 0
    bl = Blur()
    r = Rain()
    sn = SpeckleNoise()
    nf = NoFault()
    list_of_faults = [bl,r,sn,nf]
    
    countImagesInFolder(path = source_path)
    
    createFolders(faults, intensity, output_folder = True, no_fault = True)
    # load all images one by one
    for samp in glob.glob(source_path + '/*.'+ext):
        sample = Image.open(samp)
        for j in intensity:
            for faultObject in list_of_faults:
                if faultObject.fault_type != 'NoFault':
                    faultObject.intensity = j
                    faulty_image = faultObject.inject(sample)
                    if faulty_image.size != new_size:
                        faulty_image = faulty_image.resize((224, 224))
                    image_name = specifyImageName(i, faultObject.fault_type, faultObject.intensity)
                    path = current_path +'/' + output_path +'/'+faultObject.fault_type+'/' +j+'/'
                    saveImage(path, image_name, faulty_image)
                else:
                    # No Fault
                    path = current_path + '/' + output_path + '/' +faultObject.fault_type + '/'
                    image_name = specifyImageName(i, faultObject.fault_type)
                    if faulty_image.size != new_size:
                        sample = sample.resize((224, 224))
                    saveImage(path, image_name, sample)
                        
        # indication to number of samples
        if i % 100 == 0:
            pass#print(f'{i}  Samples saved', )
        i = i + 1
    print('Done saving', i,'Samples in the path', current_path + '/' + output_path)
    
def labelingfolder(fault_item, intensity_item, path):
    labels = [] # emty list to add labels information and path to data before saving it as csv file
    for path_file in glob.glob(path + '/*' + ext):   # iterate through all files
        f_path = path_file
        sep_name = path_file[len(path):-4].split('_ ') # split the file path to extract file name without extension
        nameFragment = sep_name[0].split('_')
        c_path = current_path +'/' + output_path + '/NoFault/' + nameFragment[0] + '_NoFault.jpg' # link file to the other clean no fault file with the similar iteration in name
        c_path = c_path.replace("\\", "/")
        f_path = f_path.replace("\\", "/")
        c_path = c_path.replace("//", "/")
        f_path = f_path.replace("//", "/")
        labels.append([nameFragment[0],nameFragment[1],nameFragment[2],c_path,f_path]) # add information to list
    # after iterate through all the data, now we generate the csv file
    pd.DataFrame()
    df = pd.DataFrame(labels)
    df.columns = fields
    df.head()
    df.to_csv(current_path + '/' +  output_path + '/' + fault_item + '/' + fault_item +'_'+intensity_item+"_label.csv", index=False)
    # print a message that file generated successfully
    print(fault_item+'_'+intensity_item+"_label.csv"+ " CSV file have been successfully saved")
    
def generateLabels():
    for fault in faults:
        for intensity_item in intensity:
            labelingfolder(fault, intensity_item, path = current_path + '/' +  output_path + '/' + fault + '/' + intensity_item + '/')

In [10]:
saveAllFaults(source_path)

found  600 file(s) with the extenstion ".jpg" 

The new directory /Dataset is created!
The new directory NoFault is created!
The new directory Blur is created!
The new directory l is created!
The new directory m is created!
The new directory e is created!
The new directory Rain is created!
The new directory l is created!
The new directory m is created!
The new directory e is created!
The new directory SpeckleNoise is created!
The new directory l is created!
The new directory m is created!
The new directory e is created!
Done saving 600 Samples in the path e:\transfer from uni//Dataset


In [20]:
print('\nGenerating the csv labels file:')


Generating the csv labels file:


In [21]:
generateLabels()

Blur_l_label.csv CSV file have been successfully saved
Blur_m_label.csv CSV file have been successfully saved
Blur_e_label.csv CSV file have been successfully saved
Rain_l_label.csv CSV file have been successfully saved
Rain_m_label.csv CSV file have been successfully saved
Rain_e_label.csv CSV file have been successfully saved
SpeckleNoise_l_label.csv CSV file have been successfully saved
SpeckleNoise_m_label.csv CSV file have been successfully saved
SpeckleNoise_e_label.csv CSV file have been successfully saved


## Do not excute its only for explaining

In [7]:
labelingfolder('Blur', 'l', path = current_path + '/' +  output_path + '/' + 'Blur' + '/' + 'l' + '/')

Blur_l_label.csv CSV file have been successfully saved


In [8]:
def generateLabels():
    for fault in faults:
        for intensity_item in intensity:
            labelingfolder(fault, intensity_item, path = current_path + '/' +  output_path + '/' + fault + '/' + intensity_item + '/')

# Info about FaultClass:
description to FaultClass.py code implementation
### fault intensity:
<code>&nbsp;'l': low&nbsp;</code>
<code>&nbsp;'m': medium&nbsp;</code>
<code>&nbsp;'e': extreme&nbsp;</code>
### fault types:
<code>&nbsp;Blur&nbsp;</code>
<code>&nbsp;Brightness&nbsp;</code>
<code>&nbsp;Dark&nbsp;</code>
<code>&nbsp;Specklenoise&nbsp;</code>
<code>&nbsp;Rain&nbsp;</code>
### generate fault with the constructors:

&emsp;&emsp;&emsp;&emsp;<code><b><big>Low Intensity</big></b></code>&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;<code><b><big>Medium Intensity</big></b></code>&emsp;&emsp;&emsp;&emsp;<code><b><big>Extreme Intensity</big></b></code><br>

&emsp;&emsp;&emsp;&emsp;<code>1.Blur('l')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;<code>Blur('m')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<code>Blur('e')</code><br>

&emsp;&emsp;&emsp;&emsp;<code>2. Brightness('l')</code>&emsp;&emsp;&emsp;&emsp;&ensp;<code>Brightness('m')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<code>Brightness('e')</code><br>


&emsp;&emsp;&emsp;&emsp;<code>3. Dark('l')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<code>Dark('m')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<code>Dark('e')</code><br>


&emsp;&emsp;&emsp;&emsp;<code>4. Rain('l')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<code>Rain('m')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<code>Rain('e')</code><br>


&emsp;&emsp;&emsp;&emsp;<code>5. SpeckleNoise('l')</code>&emsp;&emsp;&emsp;&ensp;<code>SpeckleNoise('m')</code>&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;<code>SpeckleNoise('e')</code><br>

### Examples:
#### Rain:
    rain_fault = Rain('m')   # create instance of rain fault with intensity medium
    faulty_image = rain_fault.inject(clean_image) # generate the faulty image with the function inject
    
#### Speckle Noise:
    noise_fault = SpeckleNoise('e')   # create instance of rain fault with intensity extreme
    faulty_image = noise_fault.inject(clean_image) # generate the faulty image with the function inject